# Azure 함수 예제

이 노트북에서는 Azure OpenAI 서비스에서 함수 호출 기능을 사용하는 방법을 보여 줍니다. 함수를 사용하면 채팅 완료 호출자가 모델이 외부 도구 및 데이터로 기능을 확장하는 데 사용할 수 있는 기능을
기능을 외부 도구 및 데이터 소스로 확장하는 데 사용할 수 있습니다.

채팅 함수에 대한 자세한 내용은 OpenAI 블로그(https://openai.com/blog/function-calling-and-other-api-updates)에서 확인할 수 있습니다.

**참고**: 채팅 기능을 사용하려면 gpt-4 및 gpt-35-turbo의 `-0613` 레이블로 시작하는 모델 버전이 필요합니다. 이전 버전의 모델에서는 지원되지 않습니다.

설정 ## 설정

먼저 필요한 종속성을 설치합니다.

In [ ]:
! pip install openai
# (Optional) If you want to use Microsoft Active Directory
! pip install azure-identity

In [3]:
import os
import openai


또한, Azure OpenAI 서비스에 제대로 접속하기 위해서는 [Azure 포털](https://portal.azure.com)에서 적절한 리소스를 생성해야 합니다(자세한 방법은 [Microsoft Docs](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal)에서 확인 가능).

리소스가 생성되면 가장 먼저 사용해야 하는 것은 엔드포인트입니다. 엔드포인트는 *"리소스 관리"* 섹션의 *"키 및 엔드포인트"* 섹션을 확인하면 얻을 수 있습니다. 이 정보가 있으면 이 정보를 사용하여 SDK를 설정합니다:

In [27]:
openai.api_base = "" # Add your endpoint here

# functions is only supported by the 2023-07-01-preview API version
openai.api_version = "2023-07-01-preview"

인증 ### 인증

Azure OpenAI 서비스는 API 키 및 Azure 자격 증명을 포함하는 여러 인증 메커니즘을 지원합니다.

In [4]:
use_azure_active_directory = False


#### API 키를 사용한 인증

OpenAI SDK가 *Azure API 키*를 사용하도록 설정하려면 `api_type`을 `azure`로 설정하고 `api_key`를 엔드포인트와 연결된 키로 설정해야 합니다(이 키는 [Azure 포털](https://portal.azure.com)의 *"리소스 관리"* 아래 "키 및 엔드포인트"*에서 찾을 수 있음).

In [ ]:
if not use_azure_active_directory:
    openai.api_type = "azure"
    openai.api_key = os.environ["OPENAI_API_KEY"]

> 참고: 이 예제에서는 코드에서 변수를 설정하여 Azure API를 사용하도록 라이브러리를 구성했습니다. 개발의 경우 환경 변수를 대신 설정하는 것을 고려하세요:

```
OPENAI_API_BASE
OPENAI_API_KEY
OPENAI_API_TYPE
OPENAI_API_VERSION
```

#### Microsoft Active Directory를 사용한 인증
이제 Microsoft Active Directory 인증을 통해 키를 얻는 방법을 살펴보겠습니다.

In [ ]:
from azure.identity import DefaultAzureCredential

if use_azure_active_directory:
    default_credential = DefaultAzureCredential()
    token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

    openai.api_type = "azure_ad"
    openai.api_key = token.token

토큰은 일정 기간 동안 유효하며 그 이후에는 만료됩니다. 모든 요청에 유효한 토큰이 전송되도록 하려면 요청.auth에 연결하여 만료되는 토큰을 새로 고칠 수 있습니다:

In [ ]:
import typing
import time
import requests

if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req

if use_azure_active_directory:
    session = requests.Session()
    session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

    openai.requestssession = session

기능 ## 기능

설정 및 인증이 완료되면 이제 Azure OpenAI 서비스에서 함수를 사용할 수 있습니다. 이 과정은 몇 가지 단계로 나뉩니다:

1. 함수 정의
2. 함수 정의를 채팅 완료 API에 전달하기
3. 응답의 인수를 사용하여 함수 호출
4. 함수 응답을 채팅 완료 API로 피드백

#### 1. 함수 정의하기

함수 이름, 선택적 설명, 함수가 허용하는 매개변수(JSON 스키마로 설명됨)를 포함하는 함수 목록을 정의할 수 있습니다.

In [21]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location"],
        },
    }
]

#### 2. 함수 정의를 채팅 완료 API에 전달합니다.

이제 함수를 채팅 완료 API에 전달할 수 있습니다. 모델이 함수를 호출해야 한다고 판단하면 선택 항목에 'function_call'의 'finish_reason'이 채워지고 호출할 함수와 그 인수에 대한 세부 정보가 '메시지'에 표시됩니다. 선택적으로 `function_call` 키워드 인수를 설정하여 모델이 특정 함수를 강제로 호출하도록 할 수 있습니다(예: `function_call={"name": get_current_weather}`). 기본적으로 이 인수는 `자동`으로 설정되어 있어 모델이 함수를 호출할지 여부를 선택할 수 있습니다.

In [22]:
messages = [
    {"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
    {"role": "user", "content": "What's the weather like today in Seattle?"}
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="gpt-35-turbo-0613",
    messages=messages,
    functions=functions,
)
print(chat_completion)

{
  "choices": [
    {
      "content_filter_results": {},
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "function_call": {
          "arguments": "{\n  \"location\": \"Seattle, WA\"\n}",
          "name": "get_current_weather"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1689702512,
  "id": "chatcmpl-7dj6GkYdM7Vw9eGn02bc2qqjN70Ps",
  "model": "gpt-4",
  "object": "chat.completion",
  "prompt_annotations": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      },
      "prompt_index": 0
    }
  ],
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 115,
    "total_to

#### 3. 응답의 인수를 사용하여 함수 호출

함수 호출의 이름은 처음에 제공된 이름이며 인수는 함수 정의에 포함된 스키마와 일치하는 JSON을 포함합니다.

In [23]:
import json

def get_current_weather(request):
    """
    This function is for illustrative purposes.
    The location and unit should be used to determine weather
    instead of returning a hardcoded response.
    """
    location = request.get("location")
    unit = request.get("unit")
    return {"temperature": "22", "unit": "celsius", "description": "Sunny"}

function_call =  chat_completion.choices[0].message.function_call
print(function_call.name)
print(function_call.arguments)

if function_call.name == "get_current_weather":
    response = get_current_weather(json.loads(function_call.arguments))

get_current_weather
{
  "location": "Seattle, WA"
}


#### 4. 채팅 완료 API에 함수 응답 피드백 피드

함수의 응답은 역할이 "함수"로 설정된 새 메시지로 직렬화되어야 합니다. 이제 모델은 응답 데이터를 사용하여 답변을 공식화합니다.

In [24]:
messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": json.dumps(response)
    }
)

function_completion = openai.ChatCompletion.create(
    deployment_id="gpt-35-turbo-0613",
    messages=messages,
    functions=functions,
)

print(function_completion.choices[0].message.content.strip())

Today in Seattle, the weather is sunny with a temperature of 22 degrees celsius.
